In [44]:
#!pip install agentpy owlready2

In [45]:
import agentpy as ap
from matplotlib import pyplot as plt
import IPython
import random
from owlready2 import *
import ast

In [46]:
onto = get_ontology("file://onto.owl")

In [47]:
#ONTOLOGIA
#onto.delete()

with onto:

    class Entity(Thing):
        pass

    class Drone(Entity):
        pass

    class Place(Thing):
        pass

    class Position(Thing):
        pass

    class Prisioner(Entity):
        pass

    class Camera(Entity):
      pass

    class Security(Entity):
      pass

    class has_place(ObjectProperty, FunctionalProperty):
        domain = [Entity]
        range = [Place]
        pass

    class has_position(DataProperty, FunctionalProperty):
        domain = [Place]
        range = [str]
        pass

    class has_existence(DataProperty, FunctionalProperty):
        domain = [Prisioner]
        range = [bool]
        pass

    class has_alert(DataProperty, FunctionalProperty):
        domain = [Camera]
        range = [bool]
        pass

In [48]:
class DroneAgent(ap.Agent):

    #Razonamiento
    def see(self, e):
      """
      Percibir el ambiente
      """
      #Agregar su propia posición self.per[0]
      own_position = str(e.positions[self])
      self.per = [Drone(has_place=Place(has_position=str(own_position)))]

      #Agregar si un prisionero existe en su campo de visión self.per[1]
      computer_vision = ["building", "duck", "nbunny"]
      if "bunny" in computer_vision:
        self.per.append(Prisioner(has_existence = True))
      else:
        self.per.append(Prisioner(has_existence = False))

      #Cámaras que perciben actividad fuera de lo normal self.per[2]
      cameras_alert = ['camera1', None, None, None]
      self.per.append(cameras_alert)
      pass

    def brf(self):
      """
      Dada su percepción del mundo y creencias actuales, modifica sus creencias
      """
      camera_names = ['camera1', 'camera2', 'camera3', 'camera4']

      self.beliefs['own_position'] = self.per[0]
      self.beliefs['seeing_prisioner'] = self.per[1]

      for i, alert in enumerate(self.per[2]):
          if i < len(camera_names):
              camera_name = camera_names[i]
              if camera_name in self.beliefs['cameras']:
                  if alert is None:
                      self.beliefs['cameras'][camera_name].has_alert = False
                  else:
                      self.beliefs['cameras'][camera_name].has_alert = True

      self.beliefs['camera_alert'] = self.choose_camera()

      pass

    def opt(self):
      """
      Define un nuevo objetivo dadas sus intenciones actuales y percepción
      """
      #Solo generar nuevos objetivos cuando vea algo sospechoso-> Nueva posición objetivo además de las que ya están guardadas
      pass

    def filter(self):
      """
      Dadas sus creencias, deseos e intenciones, elige una nueva intención
      """
      if self.first_step:
        self.intention = self.desires['path1'][1]
        self.beliefs['current_path'] = 'path1'

      for rule in self.rules:
          act = rule()
          if act is not None:
              print(f"Rule {rule} returned action: {act.__name__}")
              act()
      print("No rule applied")
      pass

    def planning(self):
      """
      Dadas una intención y un conjunto de acciones, define un plan
      """
      self.plan = self.find_path()
      pass

    def next(self):
      """
      Llamar al razonamiento y acciones del agente
      """
      self.brf()
      self.opt()
      self.filter()

      if self.first_step:
        self.first_step = False
      elif self.beliefs['seeing_prisioner'].has_existence:
        self.alert()
      elif not self.plan:
        self.planning()
      elif self.plan:
        eval(self.plan[0])
        print(self.plan[0])
        self.plan.pop(0)

      pass

    #def reconsider(self):

      #if alarm, alarm: True, self.idle and errase plan
      #new plan between two posible positions, choose the one closest to objective

      #if prisioner is present in percepto, self.idle and errase plan?
      #send alarm to police










    #Simulación de Agente
    def setup(self):
      """
      Inicialización del agente
      """
      self.agentType = 0 # Tipo de agente
      self.directionTag = 'N' # Dirección inicial
      self.direction = (-1,0) # Dirección inicial
      self.per = []
      self.index = 0

      #Mapa
      self.map = {}

      self.coordinates = {
          'camera1': (10, 3),
          'camera2': (1, 3),
          'camera3': (1, 17),
          'camera4': (10, 17)
        }

      for name, position in self.coordinates.items():
          camera = Camera(name)
          camera.has_place = Place(has_position=str(position))
          camera.has_alert = False

          self.map[name] = camera

      #Creencias del agente
      self.beliefs = {'own_position': None,
                      'seeing_prisioner': False,
                      'current_path': None,
                      'cameras': self.map,
                      'camera_alert': None}

      # Acciones del agente
      self.actions = (
          self.find_path,
          self.next_position,
          self.switch_path,
          self.move,
          self.turn,
          self.idle
      )

      #Reglas del agente
      self.rules = (
          self.rule_1,
          self.rule_2,
          self.rule_3,
          self.rule_4,
          self.rule_5,
          self.rule_6,
          self.rule_7,
          self.rule_8,
          self.rule_9,
          self.rule_10,
          self.rule_11,
          self.rule_12,
          self.rule_13,
          self.rule_14,
          self.rule_15,
          self.rule_16,
          self.rule_17,
          self.rule_18,
          self.rule_19

      )

      #Deseos del agente
      self.desires = {
          'path1': [(10,7),(10,3),(1,3),(1,17),(10, 17)],
          'path2': [(10,7),(4,7),(4,13),(10,13)],
      }

      #Intenciones del agente
      self.intention = None

      #Planes del agente
      self.plan = None

      #First step
      self.first_step = True

    def step(self):
      """
      Función paso a paso
      """
      self.see(self.model.grid) #Percepción del Grid
      self.next() #Razonamiento y acción
      pass

    def update(self):
      pass

    def end(self):
      pass














    #RULES
    def rule_1(self):
      """
      Regla deductiva para elegir la siguiente intención
      """
      # Validador de regla
      validator = [False, False, False, False]

      # Proposición 1: Si estoy en path_1
      if self.beliefs['current_path'] == 'path1':
              validator[0] = True

      # Proposición 2: Si llegué a mi posición deseada
      if self.per[0].has_place.has_position == str(self.intention):
          validator[1] = True

      # Proposición 3: No he llegado al final del camino
      if self.per[0].has_place.has_position != str(self.desires['path1'][-1]):
          validator[2] = True

      # Proposición 4: No hay alerta
      validator[3] = True
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[3] = False



      if sum(validator) == 4:
        return self.next_position

      return None

    def rule_2(self):
      """
      Regla deductiva para cambiar de camino
      """
      # Validador de regla
      validator = [False, False, False, False]

      # Proposición 1: Si estoy en path_1
      if self.beliefs['current_path'] == 'path1':
              validator[0] = True

      # Proposición 2: Si llegué a mi posición deseada
      if self.per[0].has_place.has_position == str(self.intention):
          validator[1] = True

      # Proposición 3: LLegué al final del camino
      if self.per[0].has_place.has_position == str(self.desires['path1'][-1]):
          validator[2] = True

      # Proposición 4: No hay alerta
      validator[3] = True
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[3] = False

      if sum(validator) == 4:
        return self.switch_path

      return None

    def rule_3(self):
      """
      Regla deductiva para elegir la siguiente intención
      """
      # Validador de regla
      validator = [False, False, False, False]

      # Proposición 1: Si estoy en path_2
      if self.beliefs['current_path'] == 'path2':
              validator[0] = True

      # Proposición 2: Si llegué a mi posición deseada
      if self.per[0].has_place.has_position == str(self.intention):
          validator[1] = True

      # Proposición 3: No he llegado al final del camino
      if self.per[0].has_place.has_position != str(self.desires['path2'][-1]):
          validator[2] = True

      # Proposición 4: No hay alerta
      validator[3] = True
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[3] = False

      if sum(validator) == 4:
        return self.next_position

      return None

    def rule_4(self):
      """
      Regla deductiva para cambiar de camino
      """
      # Validador de regla
      validator = [False, False, False, False]

      # Proposición 1: Si estoy en path_2
      if self.beliefs['current_path'] == 'path2':
              validator[0] = True

      # Proposición 2: Si llegué a mi posición deseada
      if self.per[0].has_place.has_position == str(self.intention):
          validator[1] = True

      # Proposición 3: Llegué al final del camino
      if self.per[0].has_place.has_position == str(self.desires['path2'][-1]):
          validator[2] = True

      # Proposición 4: No hay alerta
      validator[3] = True
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[3] = False

      if sum(validator) == 4:
        return self.switch_path

      return None

    #Estoy en pos 2, hay alerta, no detecto nada en visión computacional -> Ir a pos1
    def rule_5(self):
      """
      Regla deductiva para ir a la cámara
      """

      #Validador de regla
      validator = [False, False, False]

      # Proposición 1: Si estoy en pos2
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path2'][1]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      #Ir a posición 1
      if sum(validator) == 3:
        print("Intention changed")
        self.intention = self.desires['path2'][0]

      return None

    #Estoy en pos 3, hay alerta, no veo nada con visión computacional -> Ir a pos4
    def rule_6(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False]

      # Proposición 1: Si estoy en pos3
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path2'][2]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      #Ir a posición 4
      if sum(validator) == 3:
        print("Intention changed")
        self.intention = self.desires['path2'][3]

      return None

    #Estoy en pos 4, hay alerta, no veo nada con visión computacional, cámara está a la derecha mía -> cambiar a path 1, ir a C4
    def rule_7(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False]

      # Proposición 1: Si estoy en pos4
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path2'][3]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está a la derecha mía
      if self.beliefs['cameras']['camera3'].has_alert == True or self.beliefs['cameras']['camera4'].has_alert == True:
        validator[3] = True

      if sum(validator) == 4:
        print("Intention changed C4 :0")
        self.intention = self.desires['path1'][4]
        self.beliefs['current_path'] = 'path1'

      return None

    #Estoy en pos 4, hay alerta, no veo nada con visión computacional, cámara está a la izquierda mía -> cambiar a path 1, ir a C1
    def rule_8(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False]

      # Proposición 1: Si estoy en pos4
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path2'][3]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está a la izquierda
      if self.beliefs['cameras']['camera1'].has_alert == True or self.beliefs['cameras']['camera2'].has_alert == True:
        validator[3] = True

      if sum(validator) == 4:
        print("Intention changed")
        self.intention = self.desires['path1'][1]
        self.beliefs['current_path'] = 'path1'

      return None

    #Estoy en pos 1, hay alerta, no veo nada con visión computacional, cámara está a la derecha -> cambiar a path 1, a C4
    def rule_9(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False]

      # Proposición 1: Si estoy en pos1
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path2'][0]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está a la derecha
      if self.beliefs['cameras']['camera3'].has_alert == True or self.beliefs['cameras']['camera4'].has_alert == True:
        validator[3] = True

      if sum(validator) == 4:
        print("Intention changed")
        self.intention = self.desires['path1'][4]
        self.beliefs['current_path'] = 'path1'

      return None



    #Estoy en pos 1, hay alerta, no estoy en la posición de la cámara que mandó la alerta, no veo nada con visión computacional, cámara está a la izquierda mía -> cambiar a path 1, a C1
    def rule_10(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False]

      # Proposición 1: Si estoy en pos1
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path2'][0]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está a la izquierda
      if self.beliefs['cameras']['camera1'].has_alert == True or self.beliefs['cameras']['camera2'].has_alert == True:
        validator[3] = True

      if sum(validator) == 4:
        print("Intention changed")
        self.intention = self.desires['path1'][1]
        self.beliefs['current_path'] = 'path1'

      return None


    #Estoy en pos C1, hay alerta, no estoy en la posición de la cámara que mandó la alerta, no veo nada con visión computacional, cámara está a la derecha mía -> dejar en path 1, ir a C4
    def rule_11(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False, False]

      # Proposición 1: Si estoy en C1
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path1'][1]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está a la derecha
      if self.beliefs['cameras']['camera4'].has_alert == True:
        validator[3] = True

      # Proposición 5: No estoy en la posición de la cámara que envió la alerta
      if self.beliefs['own_position'].has_place.has_position != str(self.beliefs['camera_alert']):
        validator[4] = True

      if sum(validator) == 5:
        print("Intention changed")
        self.intention = self.desires['path1'][4]

      return None

    #Estoy en pos C1, hay alerta, no estoy en la posición de la cámara que mandó la alerta, no veo nada con visión computacional, cámara está arriba mío o cámara es C3 -> dejar en path 1, ir a C2
    def rule_12(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False, False]

      # Proposición 1: Si estoy en C1
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path1'][1]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está arriba
      if self.beliefs['cameras']['camera2'].has_alert == True or self.beliefs['cameras']['camera3'].has_alert == True:
        validator[3] = True

      # Proposición 5: No estoy en la posición de la cámara que envió la alerta
      print(str(self.beliefs['camera_alert']))
      if self.beliefs['own_position'].has_place.has_position != str(self.beliefs['camera_alert']):
        validator[4] = True

      if sum(validator) == 5:
        print("Intention changed")
        self.intention = self.desires['path1'][2]

      return None


    #Estoy en pos C4, hay alerta, no estoy en la posición de la cámara que mandó la alerta, no veo nada con visión computacional, cámara está a la izquierda mía -> dejar en path 1, ir a C1
    def rule_13(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False, False]

      # Proposición 1: Si estoy en C4
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path1'][4]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está a la izquierda
      if self.beliefs['cameras']['camera1'].has_alert == True:
        validator[3] = True

      # Proposición 5: No estoy en la posición de la cámara que envió la alerta
      print(str(self.beliefs['camera_alert']))
      if self.beliefs['own_position'].has_place.has_position != str(self.beliefs['camera_alert']):
        validator[4] = True

      if sum(validator) == 5:
        print("Intention changed")
        self.intention = self.desires['path1'][1]

      return None

    #Estoy en pos C4, hay alerta, no estoy en la posición de la cámara que mandó la alerta, no veo nada con visión computacional, cámara está arriba mío o cámara es C2-> dejar en path 1, ir a C3
    def rule_14(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False, False]

      # Proposición 1: Si estoy en C4
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path1'][4]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está arriba
      if self.beliefs['cameras']['camera2'].has_alert == True or self.beliefs['cameras']['camera3'].has_alert == True:
        validator[3] = True

      # Proposición 5: No estoy en la posición de la cámara que envió la alerta
      print(str(self.beliefs['camera_alert']))
      if self.beliefs['own_position'].has_place.has_position != str(self.beliefs['camera_alert']):
        validator[4] = True

      if sum(validator) == 5:
        print("Intention changed")
        self.intention = self.desires['path1'][3]

      return None

    #Estoy en pos C2, hay alerta, no estoy en la posición de la cámara que mandó la alerta, no veo nada con visión computacional, cámara está a la derecha mía -> dejar en path 1, ir a C3
    def rule_15(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False, False]

      # Proposición 1: Si estoy en C2
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path1'][2]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está a la derecha
      if self.beliefs['cameras']['camera3'].has_alert == True:
        validator[3] = True

      # Proposición 5: No estoy en la posición de la cámara que envió la alerta
      print(str(self.beliefs['camera_alert']))
      if self.beliefs['own_position'].has_place.has_position != str(self.beliefs['camera_alert']):
        validator[4] = True

      if sum(validator) == 5:
        print("Intention changed")
        self.intention = self.desires['path1'][3]

      return None

    #Estoy en pos C2, hay alerta, no estoy en la posición de la cámara que mandó la alerta, no veo nada con visión computacional, cámara está abajo mío o cámara es C4 -> dejar en path 1, ir a C1
    def rule_16(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False, False]

      # Proposición 1: Si estoy en C2
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path1'][2]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está a abajo
      if self.beliefs['cameras']['camera1'].has_alert == True or self.beliefs['cameras']['camera4'].has_alert == True:
        validator[3] = True

      # Proposición 5: No estoy en la posición de la cámara que envió la alerta
      print(str(self.beliefs['camera_alert']))
      if self.beliefs['own_position'].has_place.has_position != str(self.beliefs['camera_alert']):
        validator[4] = True

      if sum(validator) == 5:
        print("Intention changed")
        self.intention = self.desires['path1'][1]

      return None

    #Estoy en pos C3, hay alerta, no estoy en la posición de la cámara que mandó la alerta, no veo nada con visión computacional, cámara está a la izquierda mía -> dejar en path 1, ir a C2

    def rule_17(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False, False]

      # Proposición 1: Si estoy en C3
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path1'][3]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está a la izquierda
      if self.beliefs['cameras']['camera2'].has_alert == True:
        validator[3] = True

      # Proposición 5: No estoy en la posición de la cámara que envió la alerta
      print(str(self.beliefs['camera_alert']))
      if self.beliefs['own_position'].has_place.has_position != str(self.beliefs['camera_alert']):
        validator[4] = True

      if sum(validator) == 5:
        print("Intention changed")
        self.intention = self.desires['path1'][2]

      return None

    #Estoy en pos C3, hay alerta, no estoy en la posición de la cámara que mandó la alerta, no veo nada con visión computacional, cámara está abajo mío o cámara C1-> dejar en path 1, ir a C4
    def rule_18(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False, False, False, False, False]

      # Proposición 1: Si estoy en C3
      if self.beliefs['own_position'].has_place.has_position == str(self.desires['path1'][3]):
          validator[0] = True

      # Proposición 2: Hay alerta
      validator[1] = False
      for name, camera in self.beliefs['cameras'].items():
            if camera.has_alert == True:
              validator[1] = True

      # Proposición 3: No veo nada con visión computacional
      if not self.beliefs['seeing_prisioner'].has_existence:
        validator[2] = True

      # Proposición 4: Cámara está abajo
      if self.beliefs['cameras']['camera4'].has_alert == True or self.beliefs['cameras']['camera1'].has_alert == True:
        validator[3] = True

      # Proposición 5: No estoy en la posición de la cámara que envió la alerta
      print(str(self.beliefs['camera_alert']))
      if self.beliefs['own_position'].has_place.has_position != str(self.beliefs['camera_alert']):
        validator[4] = True

      if sum(validator) == 5:
        print("Intention changed")
        self.intention = self.desires['path1'][4]

      return None

    #Estoy en posición de la cámara que mandó la alerta y no veo nada con visión computacional -> Girar 360°
    def rule_19(self):
      """
      Regla deductiva para ir a la cámara
      """
      #Validador de regla
      validator = [False]

      # Proposición 1: Si estoy en la posición de la cámara que envió la alerta
      if self.beliefs['own_position'].has_place.has_position == str(self.beliefs['camera_alert']):
        validator[0] = True

      if sum(validator) == 1:
        print("Intention changed")
        print("Posición alcanzada")

      return None

    #Giré 360° y no veo nada -> Seguir a la siguiente posición en el path actual

    #Estoy en posición de la cámara que mandó la alerta y veo algo con visión computacional: detenerme y avisar al policia? Iniciar comunicación

    #ACTIONS
    #For planning
    def find_path(self):
        path = []
        distance = tuple(a - b for a, b in zip( self.intention, self.model.grid.positions[self]))

        # Horizontal movement
        x = distance[0]
        path.append( self.turn('x', x) )
        for i in range(0, abs(x)):
          path.append('self.move()')

        # Vertical movement
        y = distance[1]
        path.append( self.turn('y', y) )
        for i in range(0, abs(y)):
          path.append('self.move()')

        return path

    def next_position(self):
      print(self.index)
      self.index += 1
      self.intention = self.desires[self.beliefs['current_path']][self.index]

    def change_intention(self):
      print(f"Intention changed to {self.intention}")

    def switch_path(self):
      self.index = 0
      if self.beliefs['current_path'] == 'path1':
        self.beliefs['current_path'] = 'path2'
      elif self.beliefs['current_path'] == 'path2':
        self.beliefs['current_path'] = 'path1'
      self.intention = self.desires[self.beliefs['current_path']][self.index]

    def alert(self):
      self.plan = None
      self.intention = None
      self.idle()
      print("Communication with security agent")
      self.beliefs['seeing_prisioner'] = Prisioner(has_existence = False)

    def choose_camera(self):
      camera_names = ['camera1', 'camera2', 'camera3', 'camera4']
      cameras_with_alerts = []

      #Guardar todas las cámaras que tengan una alerta
      for camera in camera_names:
          if self.beliefs['cameras'][camera].has_alert:
              cameras_with_alerts.append(camera)

      #Si no hay cámaras con alertas regresa None
      if not cameras_with_alerts:
        return None

      #En caso de que haya sólo un elemento se regresa ese valor
      chosen_camera = eval(self.beliefs['cameras'][cameras_with_alerts[0]].has_place.has_position)


      #En caso de que haya más de una cámara que detectó movimiento, elegir la más cercana a mi posición
      shortest_distance = float('inf')


      if len(cameras_with_alerts) > 1:
        for i in cameras_with_alerts:

          #Restar tupla con mi posición y sumar valor absoluto
          distance = sum(abs(x) for x in tuple(a - b for a, b in zip(eval(self.beliefs['cameras'][i].has_place.has_position), self.model.grid.positions[self])))

          #Comparar con distancia anterior y reemplazar si es necesario
          if distance < shortest_distance:
            shortest_distance = distance
            chosen_camera = self.beliefs['cameras'][i].has_place.has_position

      return chosen_camera

    #For affecting the environment
    def move(self):
        if self.directionTag == 'N':
            self.model.grid.move_by(self, (-1, 0))
        if self.directionTag == 'S':
            self.model.grid.move_by(self, (1, 0))
        if self.directionTag == 'E':
            self.model.grid.move_by(self, (0, 1))
        if self.directionTag == 'W':
            self.model.grid.move_by(self, (0, -1))
        print(f"Moved to new position: {self.model.grid.positions[self]}")
        pass

    def turn(self, axis, distance):
        if axis == 'y' and distance < 0:
            return 'self.turnW()'
        elif axis == 'y' and distance > 0:
            return 'self.turnE()'
        elif axis == 'x' and distance < 0:
            return 'self.turnN()'
        elif axis == 'x' and distance > 0:
            return 'self.turnS()'
        else:
            return 'self.idle()'

    def turnN(self):
        self.direction = (-1,0) #Hacia Norte
        self.directionTag = "N"
        pass

    def turnS(self):
        self.direction = (1,0)  #Hacia Sur
        self.directionTag = "S"
        pass

    def turnE(self):
        self.direction = (0,1) #Hacia Este
        self.directionTag = "E"
        pass

    def turnW(self):
        self.direction = (0,-1) #Hacia Oeste
        self.directionTag = "W"
        pass

    def idle(self):
        pass

In [49]:

class BuildingAgent(ap.Agent):

    def setup(self):
        self.agentType = 2
        pass

    def step(self):
        pass

    def update(self):
        pass

    def end(self):
        pass

In [50]:
class PrisionModel(ap.Model):
    def setup(self):
        self.drones = ap.AgentList(self, self.p.drones, DroneAgent)
        self.buildings = ap.AgentList(self, self.p.buildings, BuildingAgent)

        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)
        self.grid.add_agents(self.drones, positions=(self.p.dpos), random=False, empty=True)
        self.grid.add_agents(self.buildings, positions=(self.p.bpos), random=False, empty=True)
        pass

    def step(self):
        self.drones.step()
        pass

    def update(self):
        pass

    def end(self):
        pass

In [51]:
parameters = {
    'M': 14,
    'N': 20,
    'steps': 50,
    'drones': 1,
    'dpos': ((1,17),),
    'buildings': 49,
    'bpos': ((2,4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13),(2, 14), (2, 15), (2, 16),
             (3,4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13),(3, 14), (3, 15), (3, 16),
             (4,4), (4,5), (5,4), (5,5), (6,4), (6,5), (7,4), (7,5),
             (4,15), (4,16), (5,15), (5,16), (6,15), (6,16), (7,15), (7,16),
             (11, 7), (11, 8), (11, 9), (11, 10), (11, 11), (11, 12), (11, 13))
}
"""
self.desires = {
    'path1': [(10,7),(10,3),(1,3),(1,17),(10, 17)],
    'path2': [(10,7),(4,7),(4,13),(10,13)],
}
"""
# model = WarehouseModel(parameters)
# results = model.run()

"\nself.desires = {\n    'path1': [(10,7),(10,3),(1,3),(1,17),(10, 17)],\n    'path2': [(10,7),(4,7),(4,13),(10,13)],\n}\n"

In [52]:
#ANIMATION
def animation_plot(model, ax):
    # Definición de atributo para tipo de agente
    agent_type_grid = model.grid.attr_grid('agentType')
    # Definición de gráfico con colores (de acuerdo al tipo de agente)
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    # Definición de título del gráfico
    ax.set_title(f"Warehouse \n Time-step: {model.t}, "
                 f"Boxes: {0}")


In [53]:
#SIMULATION

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = PrisionModel(parameters)

#Run with animation
#If you want to run it without animation then use instead:
#model.run()
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
IPython.display.HTML(animation.to_jshtml())



(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
Intention changed
No rule applied
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
Intention changed
No rule applied
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
Intention changed
No rule applied
self.turnS()
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
Intention changed
No rule applied
Moved to new position: (2, 17)
self.move()
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
No rule applied
Moved to new position: (3, 17)
self.move()
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
No rule applied
Moved to new position: (4, 17)
self.move()
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
No rule applied
Moved to new position: (5, 17)
self.move()
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
No rule applied
Moved to new position: (6, 17)
self.move()
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
(10, 3)
No rule applied
Moved to new position: (7, 17)
self.move()
(10, 3)
(